In [1]:
from random import randint, seed
from time import perf_counter

# 1 Часть

## 1.   Случайным образом сгенерировать 25 чисел в диапазоне от 1 до 50. Каждое вновь сгенерированное число использовать в качестве ключа при добавлении в ДДП. Вывести на экран получившееся дерево (любым удобным для прочтения/интерпретации способом).

In [10]:
'''
Node.insert(key) - добавление ключа
Node.find(key) - возвращает объект Node, если есть ключ, None, если нет
Node.display() - выводит дерево в консоль
Node.delete_tree() - рекурсивное удаление ключей, включая текущий
Node.get_min() - возвращает объект Node минимального ключа
Node.get_max() - возвращает объект Node максимального ключа
Node.find_prev() - возвращает объект Node первого меньшего ключа
Node.find_next() - возвращает объект Node первого большего ключа

delete_key(node, key) - удаляет все ключи с переданным значением

Для использования функций в поддеревьях, используйте Node.find(key)
Пример: tree.find(5).get_min(), где tree - корневой Node
'''

class Node:
    
    def __init__(self, key):
    
        self.key = key
        self.left = None
        self.right = None

    def insert(self, key):

        if self.key:
            if key < self.key:

                if self.left is None:
                    self.left = Node(key)

                else:
                    self.left.insert(key)

            elif key >= self.key:

                if self.right is None:
                    self.right = Node(key)

                else:
                    self.right.insert(key)

        else:
             self.key = key

    def delete_tree(self):
        if self.left is not None:
            self.left = self.left.delete_tree()
        if self.right is not None:
            self.right = self.right.delete_tree()
        return None
                
    def find(self, key):
        if key < self.key and self.left is not None:
            return self.left.find(key)
        if key > self.key and self.right is not None:
            return self.right.find(key)
        if key == self.key:
            return self
        else:
            return None

    def find_next(self):
        if self.right is not None:
            return self.right.get_min()
        else:
            return None
        
    def find_prev(self):
        if self.left is not None:
            return self.left.get_max()
        else:
            return None

    def get_min(self):
        if self.left is None:
            return self
        return self.left.get_min()

    def get_max(self):
        if self.right is None:
            return self
        return self.right.get_max()
           
    def display(self):
        lines, *_ = self._display_aux()
        for line in lines:
            print(line)
    
    def _display_aux(self):
        """Returns list of strings, width, height, and horizontal coordinate of the root."""
        # No child.
        if self.right is None and self.left is None:
            line = '%s' % self.key
            width = len(line)
            height = 1
            middle = width // 2
            return [line], width, height, middle

        # Only left child.
        if self.right is None:
            lines, n, p, x = self.left._display_aux()
            s = '%s' % self.key
            u = len(s)
            first_line = (x + 1) * ' ' + (n - x - 1) * '_' + s
            second_line = x * ' ' + '/' + (n - x - 1 + u) * ' '
            shifted_lines = [line + u * ' ' for line in lines]
            return [first_line, second_line] + shifted_lines, n + u, p + 2, n + u // 2

        # Only right child.
        if self.left is None:
            lines, n, p, x = self.right._display_aux()
            s = '%s' % self.key
            u = len(s)
            first_line = s + x * '_' + (n - x) * ' '
            second_line = (u + x) * ' ' + '\\' + (n - x - 1) * ' '
            shifted_lines = [u * ' ' + line for line in lines]
            return [first_line, second_line] + shifted_lines, n + u, p + 2, u // 2

        # Two children.
        left, n, p, x = self.left._display_aux()
        right, m, q, y = self.right._display_aux()
        s = '%s' % self.key
        u = len(s)
        first_line = (x + 1) * ' ' + (n - x - 1) * '_' + s + y * '_' + (m - y) * ' '
        second_line = x * ' ' + '/' + (n - x - 1 + u + y) * ' ' + '\\' + (m - y - 1) * ' '
        if p < q:
            left += [n * ' '] * (q - p)
        elif q < p:
            right += [m * ' '] * (p - q)
        zipped_lines = zip(left, right)
        lines = [first_line, second_line] + [a + u * ' ' + b for a, b in zipped_lines]
        return lines, n + m + u, max(p, q) + 2, n + u // 2
    
def delete(node, key):
    if node is not None:
        if key < node.key:
            node.left = delete(node.left, key)
        elif key > node.key:
            node.right = delete(node.right, key)
        else:
            if node.left is None and node.right is None:
                return None
            elif node.left is not None and node.right is None:
                node.key = node.left.key
                node.left = delete(node.left, node.left.key)
            elif node.left is None and node.right is not None:
                node.key = node.right.key
                node.right = delete(node.right, node.right.key)
            else:
                min_right = node.find_next()
                node.key = min_right.key
                node.right = delete(node.right, min_right.key)
    return node

def delete_key(node, key):
    if not node.find(key):
        print('Нет такого ключа')
        return
    while node.find(key):
        delete(node, key)

In [20]:
array = [randint(1, 50) for _ in range(3)]
tree = Node(array[0])
for i in array[1:]:
    tree.insert(i)
tree.display()

  35_ 
 /   \
17  39


In [21]:
delete_key(tree, 35)
tree.display()

  39
 /  
17  


## 2.	Из получившегося дерева получить отсортированный массив ключей размера 25.

In [6]:
def bst_sort(tree, keys): # обход в глубину, LNR - in-order traversal
    if tree is not None:
        bst_sort(tree.left, keys)
        keys.append(tree.key)
        bst_sort(tree.right, keys)

keys = []
bst_sort(tree, keys)
print(keys)

[1, 3, 3, 5, 8, 12, 14, 16, 19, 21, 21, 30, 35, 37, 37, 43, 44, 45, 45, 46, 46, 48]


## 3.	Повторить пункты 1-2 для 1.000/5.000/10.000 чисел в диапазоне:



In [7]:
def create_sort(limit):
    time = []
    for i in [1000, 5000, 10000]:
        start_time = perf_counter()
        
        array = [randint(1, limit) for _ in range(i)]
        tree = Node(array[0])
        for j in array[1:]:
            tree.insert(j)
        
        keys=[]
        bst_sort(tree, keys)
        time.append(perf_counter() - start_time)
        print(f'Заполнение и вывод в массив бинарного дерева {i} элементов, выполнилось за: {perf_counter() - start_time} сек')
    return time

## a.	от 1 до 10.000


In [8]:
ten_thousand = create_sort(10000)

Заполнение и вывод в массив бинарного дерева 1000 элементов, выполнилось за: 0.006253600120544434 сек
Заполнение и вывод в массив бинарного дерева 5000 элементов, выполнилось за: 0.03527390002273023 сек
Заполнение и вывод в массив бинарного дерева 10000 элементов, выполнилось за: 0.12871199985966086 сек


## b.	от 1 до 500

In [9]:
five_hundred = create_sort(500)

Заполнение и вывод в массив бинарного дерева 1000 элементов, выполнилось за: 0.006808700039982796 сек
Заполнение и вывод в массив бинарного дерева 5000 элементов, выполнилось за: 0.03640000009909272 сек
Заполнение и вывод в массив бинарного дерева 10000 элементов, выполнилось за: 0.10126879997551441 сек


## 4.	Сгенерировать массивы чисел размера 1.000/5.000/10.000 (сначала в диапазоне от 1 до 10.000, потом в диапазоне от 1 до 500). Произвести сортировку массивов с использованием любого алгоритма сортировки (используйте библиотечные функции python), при это не забудьте произвести замеры времени работы на каждом размере массива.

In [10]:
def builtin_sort(limit):
    time = []
    for i in [1000, 5000, 10000]:
        start_time = perf_counter()
        
        array = [randint(1, limit) for _ in range(i)]
        
        sorted(array)
        time.append(perf_counter() - start_time)
        print(f'Генерация и сортировка массива из {i} элементов от {1} до {limit}, выполнилось за: {perf_counter() - start_time} сек')
    return time

sort_ten_thousand = builtin_sort(10000)
sort_five_hundred = builtin_sort(500)

Генерация и сортировка массива из 1000 элементов от 1 до 10000, выполнилось за: 0.002154699992388487 сек
Генерация и сортировка массива из 5000 элементов от 1 до 10000, выполнилось за: 0.005688300123438239 сек
Генерация и сортировка массива из 10000 элементов от 1 до 10000, выполнилось за: 0.013344499981030822 сек
Генерация и сортировка массива из 1000 элементов от 1 до 500, выполнилось за: 0.0011965001467615366 сек
Генерация и сортировка массива из 5000 элементов от 1 до 500, выполнилось за: 0.005473099881783128 сек
Генерация и сортировка массива из 10000 элементов от 1 до 500, выполнилось за: 0.011186999967321754 сек


## 5.	Всю полученную информацию внесите в таблицу и сделайте выводы.

In [11]:
import pandas as pd

data = [five_hundred, sort_five_hundred, ten_thousand, sort_ten_thousand]
df = pd.DataFrame(columns=['1000','5000','10000'], index=[['До 500','','До 10000',''], ['Наше решение','sorted()',
                                                                                      'Наше решение', 'sorted()']], data=data)
df
print(df, '\nВывод: timsort рулит')

                           1000      5000     10000
До 500   Наше решение  0.006802  0.036395  0.101265
         sorted()      0.001191  0.005470  0.011182
До 10000 Наше решение  0.006246  0.035267  0.128699
         sorted()      0.002149  0.005684  0.013330 
Вывод: timsort рулит


# 2 Часть

## 1.	Напишите функцию, определяющую высоту дерева. Продемонстируйте работу этой функции.

In [12]:
def node_height(node):
    if not node:
        return 0
    return max(node_height(node.left), node_height(node.right)) + 1

array = [randint(1, 100) for _ in range(25)]
tree = Node(array[0])
for _ in array[1:]:
    tree.insert(_)
print(f'Высота ДДП: {node_height(tree) - 1}')
tree.display()

Высота ДДП: 8
        ____29___                             
       /         \                            
   ___13_       35_____________               
  /      \     /               \              
 _8__   13_   33        ______49_____________ 
/    \     \           /                     \
1   10    21        __39_                 __99
 \ /               /     \               /    
 5 8              37_   45_             83_   
                     \     \           /   \  
                    38    45_     ____80  95  
                             \   /            
                            47  68___         
                                     \        
                                    78        
                                   /          
                                  73          


## 2.	Сгенерируйте случайным образом ДДП, состоящее из 50 узлов, содержащих ключи в диапазоне от 1 до 25. Далее пользователь вводит любое число X. В построенной дереве производится удаление всех вершин, у которых ключ равен X, и вывод получившегося дерева, либо пользователю сообщается, что вершины с данным ключом X в дереве не существует.

In [118]:
array = [randint(1, 25) for i in range(50)]
tree = Node(array[0])
for i in array:
    tree.insert(i)
tree.display()

 _________________________________15_                                                   
/                                    \                                                  
1______________                     15_________________________________                 
               \                                                       \                
        ______11_______________                           ____________22_               
       /                       \                         /               \              
      _7__        ____________14_       ________________18_______       22_______       
     /    \      /               \     /                         \               \      
  ___3   _8     11___           14    15_____________       ____20_         ____24_     
 /    \ /  \         \                               \     /       \       /       \    
 2    4 7  8        12_                   __________17_   18_     21_     23_     24_   
  \      \  \      / 

In [119]:
x = int(input())
delete_key(tree, x)
tree.display()

0
Нет такого ключа
 _________________________________15_                                                   
/                                    \                                                  
1______________                     15_________________________________                 
               \                                                       \                
        ______11_______________                           ____________22_               
       /                       \                         /               \              
      _7__        ____________14_       ________________18_______       22_______       
     /    \      /               \     /                         \               \      
  ___3   _8     11___           14    15_____________       ____20_         ____24_     
 /    \ /  \         \                               \     /       \       /       \    
 2    4 7  8        12_                   __________17_   18_     21_     23_     24_   
  